In [30]:
import pickle
import numpy as np
from packages.text import cleaning, flat
import pandas as pd

In [31]:
file = open('pickle/CombineModel.pkl', 'rb')
model = pickle.load(file)
file.close()

file = open('pickle/vectorizer.pkl', 'rb')
vectorizer = pickle.load(file)
file.close

<function BufferedReader.close>

In [32]:
def textclean(x):
    print(f'Text Input: {x}')
    clean = cleaning(x)
    vec_inputs = vectorizer.transform([clean])
    
    return calculatemodel(vec_inputs)

def calculatemodel(x):
    vec_inputs = x
    LRpred = model['LRmodel'].predict(vec_inputs)
    SVCpred = model['SVCmodel'].predict(vec_inputs)
    BNBpred = model['BNBmodel'].predict(vec_inputs)
    
    LRpred_conf = max(flat(model['LRmodel'].predict_proba(vec_inputs)))
    SVCpred_conf = max(flat(model['SVCmodel'].predict_proba(vec_inputs)))
    BNBpred_conf = max(flat(model['BNBmodel'].predict_proba(vec_inputs)))
    
    result = np.concatenate((LRpred, SVCpred, BNBpred))
    result_conf = [LRpred_conf, SVCpred_conf, BNBpred_conf]
    
    result = pd.DataFrame({
    'model': ['Logistic Reg', 'SVM', 'NB'],
    'predict': result,
    'confidence': result_conf
    })
    
    print(f'{result}')
    
    return pickmodel(result)

def pickmodel(x):
    result_pred = x.predict.mode()[0]
    confidence = round(x[x['predict'] == x.predict.mode()[0]]['confidence'].max()*100,2)
    print(f"Sentiment: {result_pred}")
    print(f"Confidence Level: {confidence}%\n")

In [33]:
inputs = ["happy birthday!!! @obama", "@d0n4ldTrump genshin is such a boring games"]
    
for i in inputs:
    textclean(i)

Text Input: happy birthday!!! @obama
          model   predict  confidence
0  Logistic Reg  POSITIVE    0.985974
1           SVM  POSITIVE    0.937025
2            NB  POSITIVE    0.990857
Sentiment: POSITIVE
Confidence Level: 99.09%

Text Input: @d0n4ldTrump genshin is such a boring games
          model   predict  confidence
0  Logistic Reg  NEGATIVE    0.843485
1           SVM  NEGATIVE    0.771867
2            NB  NEGATIVE    0.711912
Sentiment: NEGATIVE
Confidence Level: 84.35%

